In [ ]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd  import numpy as np  import matplotlib.pyplot as plt  import seaborn as sns  import warnings
warnings.filterwarnings('ignore')
seed = 42
np.random.seed(seed)
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import (
    Embedding, LSTM, Bidirectional, Dense, Dropout, 
    GlobalMaxPooling1D, Conv1D, SpatialDropout1D
)
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
tf.random.set_seed(seed)
keras.utils.set_random_seed(seed)
from sklearn.metrics import matthews_corrcoef, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
pd.set_option('display.max_rows', 36)
pd.set_option("display.max_colwidth", 150)


In [ ]:
MAX_WORDS = 10000  MAX_LEN = 200  EMBEDDING_DIM = 100  LSTM_UNITS = 128  DENSE_UNITS = 64  DROPOUT_RATE = 0.5  SPATIAL_DROPOUT = 0.2  BATCH_SIZE = 32
EPOCHS = 50  VALIDATION_SPLIT = 0.2
LEARNING_RATE = 1e-3


In [ ]:
train = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/train.csv", index_col="row_id")
train.head(10)

In [ ]:
train['text_length'] = train['text'].apply(lambda x: len(str(x).split()))
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.hist(train['text_length'], bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribución de Longitud de Textos')
plt.xlabel('Número de palabras')
plt.ylabel('Frecuencia')
plt.axvline(train['text_length'].mean(), color='red', linestyle='--', label=f'Media: {train["text_length"].mean():.1f}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.subplot(1, 3, 2)
train.groupby('spam_label')['text_length'].hist(bins=30, alpha=0.7, label=['Not SPAM', 'SPAM'])
plt.title('Longitud por Clase')
plt.xlabel('Número de palabras')
plt.ylabel('Frecuencia')
plt.legend()
plt.grid(True, alpha=0.3)
plt.subplot(1, 3, 3)
sns.boxplot(data=train, x='spam_label', y='text_length')
plt.title('Boxplot Longitud por Clase')
plt.xlabel('Clase (0=Not SPAM, 1=SPAM)')
plt.ylabel('Número de palabras')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
X_train_text = train['text'].values
y_train = train['spam_label'].values
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")  tokenizer.fit_on_texts(X_train_text)
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_pad, y_train, 
    test_size=VALIDATION_SPLIT, 
    random_state=seed,
    stratify=y_train  )


In [ ]:
def build_model():
    model = Sequential([
        Embedding(
            input_dim=MAX_WORDS,
            output_dim=EMBEDDING_DIM,
            input_length=MAX_LEN,
            name='embedding'
        ),
        SpatialDropout1D(SPATIAL_DROPOUT),
        Bidirectional(LSTM(LSTM_UNITS, return_sequences=True), name='bidirectional_lstm'),
        GlobalMaxPooling1D(),
        Dense(DENSE_UNITS, activation='relu', name='dense_1'),
        Dropout(DROPOUT_RATE),          Dense(1, activation='sigmoid', name='output')
    ], name='spam_classifier_v1')
    return model
model = build_model()
model.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=1e-4),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)
model.summary()


In [ ]:
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,          restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_spam_model.keras',
        monitor='val_loss',
        save_best_only=True,          verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,          patience=3,
        min_lr=1e-6,
        verbose=1
    )
]
history = model.fit(
    X_train_final, y_train_final,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,      validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)


In [ ]:
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()
mcc_score = matthews_corrcoef(y_val, y_pred)
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Not SPAM', 'SPAM'],
            yticklabels=['Not SPAM', 'SPAM'])
plt.title(f'Confusion Matrix (MCC: {mcc_score:.4f})')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
def plot_learning_curves(history, title_prefix=""):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes[0, 0].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0, 0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[0, 0].set_title(f'{title_prefix} Loss', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 1].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    axes[0, 1].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    axes[0, 1].set_title(f'{title_prefix} Accuracy', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    axes[1, 0].plot(history.history['precision'], label='Train Precision', linewidth=2)
    axes[1, 0].plot(history.history['val_precision'], label='Val Precision', linewidth=2)
    axes[1, 0].set_title(f'{title_prefix} Precision', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 1].plot(history.history['recall'], label='Train Recall', linewidth=2)
    axes[1, 1].plot(history.history['val_recall'], label='Val Recall', linewidth=2)
    axes[1, 1].set_title(f'{title_prefix} Recall', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
plot_learning_curves(history, title_prefix="Model")
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
if abs(final_val_loss - final_train_loss) < 0.1:
elif final_val_loss > final_train_loss:
else:


In [ ]:
test = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/test.csv", index_col="row_id")
test.head()

In [ ]:
X_test_text = test['text'].values
X_test_seq = tokenizer.texts_to_sequences(X_test_text)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')
y_pred_proba_test = model.predict(X_test_pad, batch_size=BATCH_SIZE)
y_pred_test = (y_pred_proba_test > 0.5).astype(int).flatten()


In [ ]:
submission = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/sample_submission.csv")
submission["spam_label"] = y_pred_test
submission.to_csv('submission.csv', index=False)


In [ ]:
submission.head(10)

In [ ]:
metrics_summary = pd.DataFrame({
    'Métrica': ['MCC', 'Accuracy', 'Precision', 'Recall', 'Loss'],
    'Training': [
        'N/A',          f"{history.history['accuracy'][-1]:.4f}",
        f"{history.history['precision'][-1]:.4f}",
        f"{history.history['recall'][-1]:.4f}",
        f"{history.history['loss'][-1]:.4f}"
    ],
    'Validation': [
        f"{mcc_score:.4f}",
        f"{history.history['val_accuracy'][-1]:.4f}",
        f"{history.history['val_precision'][-1]:.4f}",
        f"{history.history['val_recall'][-1]:.4f}",
        f"{history.history['val_loss'][-1]:.4f}"
    ]
})
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(['Accuracy', 'Precision', 'Recall']))
width = 0.35
train_vals = [
    history.history['accuracy'][-1],
    history.history['precision'][-1],
    history.history['recall'][-1]
]
val_vals = [
    history.history['val_accuracy'][-1],
    history.history['val_precision'][-1],
    history.history['val_recall'][-1]
]
bars1 = ax.bar(x - width/2, train_vals, width, label='Training', alpha=0.8)
bars2 = ax.bar(x + width/2, val_vals, width, label='Validation', alpha=0.8)
ax.set_xlabel('Métrica', fontweight='bold')
ax.set_ylabel('Score', fontweight='bold')
ax.set_title(f'Comparación Métricas - Train vs Validation\nMCC Validation: {mcc_score:.4f}', 
             fontweight='bold', fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(['Accuracy', 'Precision', 'Recall'])
ax.legend()
ax.set_ylim([0, 1.1])
ax.grid(True, alpha=0.3, axis='y')
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
metrics_summary = pd.DataFrame({
    'Métrica': [
        'Matthews Correlation Coefficient',
        'Validation Accuracy',
        'Validation Precision',
        'Validation Recall',
        'Training Loss',
        'Validation Loss',
        'Total Parameters',
        'Training Samples',
        'Validation Samples',
        'Test Samples'
    ],
    'Valor': [
        f"{mcc_score:.4f}",
        f"{final_val_acc:.4f}",
        f"{history.history['val_precision'][-1]:.4f}",
        f"{history.history['val_recall'][-1]:.4f}",
        f"{final_train_loss:.4f}",
        f"{final_val_loss:.4f}",
        f"{model.count_params():,}",
        f"{len(X_train_final):,}",
        f"{len(X_val):,}",
        f"{len(test):,}"
    ]
})
